In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from sklearn.metrics import pairwise_distances
from scipy.spatial import distance_matrix

#from sklearn import cluster
#from sklearn.preprocessing import StandardScaler
%matplotlib inline

## Entrada:
**La entrada consiste de tres bases de datos:**

* Metabolismo central

* Genomas

* MIBiG


In [23]:
cols_names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore']

df = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_EF_vs_GenomeDB/pscplos17.blast",
                  sep = "\t", names = cols_names, engine = 'python')

## Definición de constantes
**En las siguientes celdas vamos a definir todas las constantes que necesitaremos**
* Diccionarios con los nombres de los genomas y de las familias de enzimas


In [35]:
#Función para filtrar por bitscore, devuelve un dataframe con las entradas que tengan 
#bitscore >= threshold
def bitscore_filter(data, threshold):
    data.query("bitscore >= @threshold", inplace = True)
    data.reset_index(drop=True, inplace=True)
    return data

In [38]:
#df ahora contiene solo aquellas entradas con bitscore >= threshold
bitscore_filter(df,100)

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...
277,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146852.774|6666666.146852|NC_002935...,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0
278,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146872.438|6666666.146872|NC_017803...,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0
279,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.111572.2478|6666666.111572|NC_00948...,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0
280,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0


,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
count,282.000000,282.000000,282.000000,282.000000,282.000000,282.000000,282.000000,282.000000,2.820000e+02,282.000000
mean,43.954397,326.652482,164.074468,3.719858,34.010638,353.560284,38.074468,359.815603,9.598846e-25,265.985816
std,12.847797,99.297926,36.823685,2.656402,33.847837,80.258290,44.150077,85.935762,6.118800e-24,196.445260
min,28.210000,169.000000,69.000000,0.000000,1.000000,280.000000,1.000000,260.000000,0.000000e+00,100.000000
25%,34.112500,258.000000,142.000000,1.000000,5.000000,306.000000,5.000000,306.000000,6.000000e-165,115.000000
50%,38.000000,291.000000,159.000000,4.000000,22.500000,316.000000,23.000000,320.000000,3.000000e-40,150.000000
75%,55.430000,368.000000,180.000000,6.000000,57.000000,370.000000,63.750000,374.000000,3.750000e-28,473.000000
max,81.600000,532.000000,280.000000,12.000000,146.000000,530.000000,302.000000,662.000000,8.000000e-23,847.000000
